In [23]:
# Erick Salas, Ricardo Whaibe, Max Escobar, Mariana Pena, Uriel Castenda, Aramis Quiroz
# https://github.com/ermiry/modelos-p3
import numpy as np
import pandas as pd
from scipy.optimize import linprog

In [24]:
# descargamos los datos del GitHub
datos = pd.read_csv (
	"https://raw.githubusercontent.com/ermiry/modelos-p3/main/transbordo.csv"
)

# cargamos los datos en una matriz
matrix = datos.to_numpy ()
print ("Data: ")
print (matrix)

Data: 
[[1000    0    0    3    4    0    0    0    0]
 [1200    0    0    2    5    0    0    0    0]
 [   0    0    0    0    7    8    6    0    0]
 [   0    0    0    0    0    0    4    9    0]
 [   0    0    0    0    0    0    5    0  800]
 [   0    0    0    0    0    0    0    3  900]
 [   0    0    0    0    0    0    0    0  500]]


In [25]:
rows, cols = matrix.shape

# sacamos las coordenadas de las celdas donde el valor es diferente de 0
coords = []
for i in range (rows):
  j = 1
  while (j < cols - 1):
    if (matrix[i][j] > 0):
      coords.append ((i, j))
    j += 1

print (coords)


[(0, 3), (0, 4), (1, 3), (1, 4), (2, 4), (2, 5), (2, 6), (3, 6), (3, 7), (4, 6), (5, 7)]


In [26]:
# creamos una matriz auxiliar para trabajar sobre ella
aux = np.zeros ((rows + 1, len (coords) + 1), dtype=int)

k = 0
for i in range (rows):
  j = 1
  while (j < cols - 1):
    if (matrix[i][j] != 0):
      aux[0][k] = matrix[i][j]
      k += 1
    j += 1

print (aux)


[[3 4 2 5 7 8 6 4 9 5 3 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]]


In [27]:
rows_aux, cols_aux = aux.shape

# agregamos restricciones a la matriz auxiliar
for i in range (rows):
  j = 1

  # checar si la oferta es mayor a 0
  if (matrix[i][0] > 0):
    aux[i + 1][cols_aux - 1] = matrix[i][0]
    j = 1
    while (j < cols - 1):
      if matrix[i][j] > 0:
        aux[i + 1][coords.index ((i, j))] = 1
      j += 1

  # la oferta es 0
  else:
    aux[i + 1][cols_aux - 1] = matrix[i][cols - 1]
    j = 1
    while (j < cols - 1):
      if (matrix[i][j] > 0):
        aux[i + 1][coords.index ((i, j))] = -1
      j += 1
    j = 0
    while (j < rows):
      if (matrix[j][i + 1] > 0):
        aux[i + 1][coords.index ((j, i + 1))] = 1
      j += 1

print (aux)

[[   3    4    2    5    7    8    6    4    9    5    3    0]
 [   1    1    0    0    0    0    0    0    0    0    0 1000]
 [   0    0    1    1    0    0    0    0    0    0    0 1200]
 [   1    0    1    0   -1   -1   -1    0    0    0    0    0]
 [   0    1    0    1    1    0    0   -1   -1    0    0    0]
 [   0    0    0    0    0    1    0    0    0   -1    0  800]
 [   0    0    0    0    0    0    1    1    0    1   -1  900]
 [   0    0    0    0    0    0    0    0    1    0    1  500]]


In [28]:
# sacamos los datos de la matriz auxiliar
objetivo = aux[0, 0:-1]
restricciones = aux[1:, :-1]
limites = aux[1:, -1]

print ("Coeficientes de funcion objetivo: ", objetivo)
print ("Coeficientes de restricciones:\n", restricciones)
print ("Limites:\n", limites)

Coeficientes de funcion objetivo:  [3 4 2 5 7 8 6 4 9 5 3]
Coeficientes de restricciones:
 [[ 1  1  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  1  0  0  0  0  0  0  0]
 [ 1  0  1  0 -1 -1 -1  0  0  0  0]
 [ 0  1  0  1  1  0  0 -1 -1  0  0]
 [ 0  0  0  0  0  1  0  0  0 -1  0]
 [ 0  0  0  0  0  0  1  1  0  1 -1]
 [ 0  0  0  0  0  0  0  0  1  0  1]]
Limites:
 [1000 1200    0    0  800  900  500]


In [29]:
nodes = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]

# calculamos la funcion simplex usando linprog
result = linprog (objetivo, A_eq=restricciones, b_eq=limites, method="simplex")

print ("Resultado:", result.fun)

for node in coords:
  print (f"{nodes[node[0]]}{nodes[node[1] - 1]} = {result.x[coords.index (node)]}")

Resultado: 20700.0
AC = 0.0
AD = 1000.0
BC = 1200.0
BD = 0.0
CD = 0.0
CE = 800.0
CF = 400.0
DF = 1000.0
DG = 0.0
EF = 0.0
FG = 500.0


/tmp/ipykernel_6453/961717917.py:4: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  result = linprog (objetivo, A_eq=restricciones, b_eq=limites, method="simplex")
